In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F

/opt/conda/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [4]:
def VAE_loss_function(recon_x, x, mu, logvar, reconst_loss='mse', a_RECONST=1., a_KLD=1., x_dim=640):
    """Loss function for VAE which consists of reconstruction and KL divergence losses.
    Thanks to https://github.com/pytorch/examples/blob/master/vae/main.py

    You can also balance weights for each loss, just to see what if KLD loss is stronger, etc.

    Args:
        reconst_loss: Reconstruction loss calculation: 'mse' or 'bce'
        a_RECONST: Weight for reconstruction loss.
        a_KLD: Weight for KLD loss.
    """

    func = (F.mse_loss if reconst_loss == 'mse'
            else F.binary_cross_entropy if reconst_loss == 'bce'
            else 'Unknown reconst_loss')
    RECONST = func(recon_x, x.view(-1, x_dim), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return RECONST*a_RECONST + KLD*a_KLD

class DenseVAE(pl.LightningModule):
    def __init__(self, x_dim, z_dim, h_dim):
        super().__init__()
            
        self.encoder = nn.Sequential(
            nn.Linear(x_dim, h_dim),
            nn.ReLU(),
            nn.Linear(h_dim, h_dim),
            nn.ReLU(),
        ) 
        self.fc_mu = nn.Linear(h_dim, z_dim)
        self.fc_logvar = nn.Linear(h_dim, z_dim)
        self.decoder = nn.Sequential (
            nn.Linear(z_dim, h_dim),
            nn.ReLU(),
            nn.Linear(h_dim, h_dim),
            nn.ReLU(),
            nn.Linear(h_dim, x_dim),
            nn.Sigmoid()
        )
        self.loss_fn = nn.MSELoss(reduction='sum')
        
    def forward(self, x):
        z_pre = self.encoder(x)
        
        mu = self.fc_mu(z_pre)
        logvar = self.fc_logvar(z_pre)
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std
        
        x_hat = self.decoder(z)
        return x_hat, z, mu, logvar
    
    
    def training_step(self, batch, batch_idx):
        x = batch["input"]
        x_recon, z, mu, logvar = self(x)
        loss = VAE_loss_function(recon_x=x_recon, x=x, mu=mu, logvar=logvar,
                                 reconst_loss='mse',
                                 a_RECONST=1.,
                                 a_KLD=.01)
        self.log("train/loss", loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x = batch["input"]
        x_recon, z, mu, logvar = self(x)
        
        loss = self.loss_fn(x, x_recon)
        self.log("val/loss", loss)

    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(), lr=1e-3)
        return opt

class DenseAE(pl.LightningModule):
    def __init__(self, x_dim, z_dim, h_dim):
        super().__init__()
            
        self.encoder = nn.Sequential(
            nn.Linear(x_dim, h_dim),
            nn.ReLU(),
            nn.Linear(h_dim, h_dim),
            nn.ReLU(),
            nn.Linear(h_dim, z_dim),
            nn.ReLU(),
        ) 
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, h_dim),
            nn.ReLU(),
            nn.Linear(h_dim, h_dim),
            nn.ReLU(),
            nn.Linear(h_dim, x_dim)
        )
        self.loss_fn = nn.MSELoss(reduction='sum')
        
    def forward(self, x):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat, z
    
    
    def training_step(self, batch, batch_idx):
        x = batch["input"]
        x_recon, _ = self(x)
        
        loss = self.loss_fn(x, x_recon)
        self.log("train/loss", loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x = batch["input"]
        x_recon, _ = self(x)
        
        loss = self.loss_fn(x, x_recon)
        self.log("val/loss", loss)

    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(), lr=5e-4)
        return opt

In [6]:
from src.data.data_module import SliderDataModule

dm = SliderDataModule(
    data_dir="../data/raw",
    batch_size=256,
    num_workers=8,
    normalize=True,
    maxlen=312,
    use_cnn=False,
    iter_over_cols=True
)
dm.setup_subset("dev", "00")
model = DenseVAE(x_dim=640, h_dim=400, z_dim=20)
early_stopping = pl.callbacks.EarlyStopping('val/loss', patience=20, min_delta=0.5)
callbacks = [early_stopping]
logger = pl.loggers.TensorBoardLogger("lightning_logs/", name="DenseVAE", sub_dir=None, version=0)
trainer = pl.Trainer(accelerator="gpu", devices=1,
                    callbacks=callbacks, max_epochs=20,
                    log_every_n_steps=1,
                    logger=logger)
trainer.fit(model, datamodule=dm)

Building index for spectrogram columns:   0%|          | 0/968 [00:00<?, ?it/s]

Building index for spectrogram columns:   0%|          | 0/456 [00:00<?, ?it/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | encoder   | Sequential | 416 K 
1 | fc_mu     | Linear     | 8.0 K 
2 | fc_logvar | Linear     | 8.0 K 
3 | decoder   | Sequential | 425 K 
4 | loss_fn   | MSELoss    | 0     
-----------------------------------------
858 K     Trainable params
0         Non-trainable params
858 K     Total params
3.433     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [8]:
from tqdm.notebook import tqdm
import numpy as np
from sklearn.metrics import roc_auc_score
y_pred = []
y_test = []
mse = nn.MSELoss(reduction="none")
for machine_id in ["00"]:#dm.machine_ids["dev"][:1]:
    print(f"Reconstruction on test set for machine {machine_id}")
    # model = models[f"dev_{machine_id}"]
    model = model.cuda().eval()
    # dm.setup_subset("dev", machine_id)
    
    for batch in tqdm(dm.test_dataloader()):
        x_test = batch['input'].cuda()
        labels = np.array(batch['label'])
        y_test.append(np.where(labels == "anomaly", 1, 0))
        with torch.no_grad():
            x_recon, _, _, _ = model(x_test)
            y_pred.append(mse(x_recon, x_test).mean(dim=1))
            
y_pred = torch.cat(y_pred).cpu().numpy()
y_test = np.concatenate(y_test)
roc_auc_score(y_test, y_pred), roc_auc_score(y_test, y_pred, max_fpr=0.1)


Reconstruction on test set for machine 00


  0%|          | 0/551 [00:00<?, ?it/s]

(0.8623470657260006, 0.6696783198995611)

In [10]:
mse(x_recon, x_test).shape

torch.Size([256, 128])